# APIs in python, fetching data through REST

In [ ]:
import requests

# Base URL for testing
BASE_URL = "https://jsonplaceholder.typicode.com/posts"

In [ ]:
# ----------------- 1. CREATE (POST) -----------------
new_post = {
    "title": "My First Post",
    "body": "This is a test post created using Python.",
    "userId": 1
}

response = requests.post(BASE_URL, json=new_post)
print("POST Response: ", response.status_code)
print(response.json())

POST Response:  201
{'title': 'My First Post', 'body': 'This is a test post created using Python.', 'userId': 1, 'id': 101}


In [ ]:
# ----------------- 2. READ (GET) -----------------
post_id = 1
response = requests.get(f"{BASE_URL}/{post_id}")
print("GET Response: ", response.status_code)
print(response.json())

GET Response:  200
{'userId': 1, 'id': 1, 'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit', 'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto'}


In [ ]:
# ----------------- 3. UPDATE (PUT) -----------------
updated_post = {
    "id": post_id,
    "title": "Updated Title",
    "body": "Updated body of the post.",
    "userId": 1
}

response = requests.put(f"{BASE_URL}/{post_id}", json=updated_post)
print("PUT (Update) Response: ", response.status_code)
print(response.json())

PUT (Update) Response:  200
{'id': 1, 'title': 'Updated Title', 'body': 'Updated body of the post.', 'userId': 1}


In [ ]:
# ----------------- 4. DELETE -----------------
response = requests.delete(f"{BASE_URL}/{post_id}")
print("DELETE Response: ", response.status_code)
print("Deleted" if response.status_code == 200 else "Failed to delete")

DELETE Response:  200
Deleted


# Databases in python, data loading and processing

In [ ]:
!pip install sqlalchemy

In [ ]:
from sqlalchemy import create_engine, text
engine = create_engine('sqlite:///census_nyc.sqlite')
connection = engine.connect()

result = connection.execute(text("SELECT * FROM census LIMIT 5"))

for row in result:
    print(row)

('Illinois', 'M', 0, 89600, 95012)
('Illinois', 'M', 1, 88445, 91829)
('Illinois', 'M', 2, 88729, 89547)
('Illinois', 'M', 3, 88868, 90037)
('Illinois', 'M', 4, 91947, 91111)


In [ ]:
from sqlalchemy import inspect

inspector = inspect(engine)
tables = inspector.get_table_names()

print("Tables:", tables)

Tables: ['census', 'state_fact']


In [ ]:
from sqlalchemy import MetaData, Table

metadata = MetaData()
census = Table('census', metadata, autoload_with=engine)

print(repr(census))

Table('census', MetaData(), Column('state', VARCHAR(length=30), table=<census>), Column('sex', VARCHAR(length=1), table=<census>), Column('age', INTEGER(), table=<census>), Column('pop2000', INTEGER(), table=<census>), Column('pop2008', INTEGER(), table=<census>), schema=None)


In [ ]:
from sqlalchemy import text
import pandas as pd

stmt = text("SELECT * FROM census")

result_proxy = connection.execute(stmt)
rows = result_proxy.fetchall()
columns = result_proxy.keys()

df = pd.DataFrame(rows, columns=columns)

df.head()

,state,sex,age,pop2000,pop2008
0,Illinois,M,0,89600,95012
1,Illinois,M,1,88445,91829
2,Illinois,M,2,88729,89547
3,Illinois,M,3,88868,90037
4,Illinois,M,4,91947,91111


In [ ]:
from sqlalchemy import Table, MetaData

metadata = MetaData()
census = Table('census', metadata, autoload_with=engine)


In [ ]:
from sqlalchemy import insert
# Define the row you want to insert
new_data = {
    'state': 'Illinois',
    'sex': 'M',
    'age': 30,
    'pop2000': 50000,
    'pop2008': 50000
}
stmt = insert(census).values(new_data)
# Execute the insert
with engine.connect() as conn:
    conn.execute(stmt)
    conn.commit()
print("Row inserted successfully.")

Row inserted successfully.


In [ ]:
stmt = text("SELECT * FROM census WHERE pop2000 = 50000")

with engine.connect() as conn:
    result = conn.execute(stmt)
    row = result.fetchall()

print(row)

[]


In [ ]:
stmt = text("DELETE FROM census WHERE pop2000 = 50000")

with engine.connect() as conn:
    conn.execute(stmt)
    conn.commit()

print("Row deleted successfully.")

Row deleted successfully.
